# 할리스 매장정보 웹스크래핑 실습

In [ ]:
# 1. 필요한 라이브러리 설치 및 임포트
!pip install requests beautifulsoup4 pandas

## 0. 웹 페이지 분석하기
- 웹 크롤링을 시작하기 전에 대상 웹사이트의 구조를 분석해야 합니다.

### 0.1 URL 엔드포인트 분석
- **대상 URL**: `https://www.hollys.co.kr/store/korea/korStore2.do`
- **HTTP 메서드**: GET
- **설명**: 할리스 커피 매장 정보를 제공하는 페이지

### 0.2 요청 파라미터 분석
- **pageNo**: 페이지 번호 (1부터 시작)
  - 예시: `?pageNo=1`, `?pageNo=2`
  - 각 페이지마다 여러 매장 정보를 포함

### 0.3 HTML 구조 분석 (개발자 도구 활용)

#### 브라우저 개발자 도구 사용법:
1. 웹 페이지 접속 후 개발자 도구 열기
    - **Windows**: `F12` 또는 `Ctrl + Shift + I` 또는 `우클릭 > 검사`
    - **Mac**: `Cmd + Option + I` 또는 `우클릭 > 검사`
2. Elements 탭에서 원하는 데이터 위치를 찾아 HTML 구조 확인

#### 매장 정보 테이블 구조:
```html
<table class="tb_store">
    <thead>
        <tr>
            <th>지역</th>
            <th>매장명</th>
            <th>영업상태</th>
            <th>주소</th>
            <th>서비스</th>
            <th>전화번호</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>서울</td>
            <td>강남점</td>
            <td>영업중</td>
            <td>서울시 강남구 ...</td>
            <td><img alt="와이파이" /></td>
            <td>02-1234-5678</td>
        </tr>
        <!-- 더 많은 매장 정보 ... -->
    </tbody>
</table>
```

### 0.4 CSS 선택자 경로
- **테이블 전체**: `table.tb_store`
- **모든 매장 행**: `table.tb_store > tbody > tr`
- **개별 셀 구조**:
  - `td:nth-child(1)`: 지역
  - `td:nth-child(2)`: 매장명
  - `td:nth-child(3)`: 영업상태
  - `td:nth-child(4)`: 주소
  - `td:nth-child(5)`: 서비스 (이미지)
  - `td:nth-child(6)`: 전화번호

### 0.5 서비스 아이콘 분석
- 서비스 정보는 `<img>` 태그의 `alt` 속성에 저장
- 예시: `<img alt="와이파이" />`, `<img alt="주차" />`
- 여러 서비스가 있을 경우 공백으로 결합

### 0.6 요청 헤더 설정
- **User-Agent** 설정 필요
  - 일부 웹사이트는 User-Agent가 없는 요청을 차단
  - 실제 브라우저처럼 보이도록 설정

## 1. 웹 페이지 요청하기
- 웹사이트에서 데이터를 가져오기

In [159]:
import requests

def get_aitimes_article_list(page: int, total: int = 28654) -> str:
    """
    AI타임즈 뉴스 AJAX 페이지의 JSON/HTML을 가져오는 함수
    """
    url = "https://www.aitimes.com/news/ajaxArticlePaging.php"
    params = {
        "total": str(total),          # ✅ total 값 추가
        "list_per_page": "20",
        "page_per_page": "10",
        "page": str(page),
        "view_type": "sm",
        "box_idxno": "0"
    }

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                      'AppleWebKit/537.36 (KHTML, like Gecko) '
                      'Chrome/141.0.0.0 Safari/537.36',
        'Referer': f'https://www.aitimes.com/news/articleList.html?page={page}',
        'Accept': 'application/json, text/javascript, */*; q=0.01',  # ✅ JSON 응답 고려
        'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
        'X-Requested-With': 'XMLHttpRequest',  # ✅ AJAX 요청임을 명시
    }

    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()
        return response.text  # JSON 문자열 반환
    except requests.exceptions.RequestException as e:
        print(f"페이지 {page} 요청 중 에러 발생: {e}")
        return ""



In [160]:
html = get_aitimes_article_list(1)
print(html[:1000])


{"result":"success","msg":"","data":[{"idxno":"203546","title":"\uacfc\uae30\ubd80, APEC \ud55c-\ubbf8 \uc815\uc0c1\ud68c\ub2f4\uc11c AI\uae30\uc220 \ud611\ub825 MOU \uccb4\uacb0","shoulder_title":"","sub_title":"","body":"(\uc0ac\uc9c4=\uacfc\uae30\ubd80)        \uacfc\ud559\uae30\uc220\uc815\ubcf4\ud1b5\uc2e0\ubd80(\uc7a5\uad00 \ubc30\uacbd\ud6c8)\ub294 APEC \uc8fc\uac04\uc5d0 \uc5f4\ub9b0 \ud55c-\ubbf8 \uc815\uc0c1\ud68c\ub2f4\uc758 \uc131\uacfc\ub85c \uc591\uad6d \uc815\ubd80 \uac04 MOU\ub97c \uccb4\uacb0\ud588\ub2e4\uace0 29\uc77c \ubc1d\ud614\ub2e4.    \ucd5c\uadfc \uc778\uacf5\uc9c0\ub2a5(AI), \uc591\uc790 \ucef4\ud4e8\ud305, \ud569\uc131\uc0dd\ubb3c\ud559 \ub4f1 \ud575\uc2ec \uc2e0\ud765 \uae30\uc220\uc758 \uc911\uc694\uc131\uc774 \ub0a0\ub85c \ucee4\uc9d0\uc5d0 \ub530\ub77c, \uc804\ubc29\uc704\uc801 \uae30\uc220 \ud611\ub825\uccb4\uacc4\ub97c \uac15\ud654\ud558\uae30 \uc704\ud574 \uc774\ubc88 MOU\ub97c \uccb4\uacb0\ud588\ub2e4\ub294 \uc124\uba85\uc774\ub2e4.    \uc591\uad6d\uc

## 2. HTML 파싱하기
- BeautifulSoup를 사용하여 HTML에서 필요한 정보를 추출

In [ ]:
def parse_store_info(html: str) -> List[Dict]:
    """
    HTML에서 매장 정보를 파싱하는 함수
    
    Args:
        html (str): 파싱할 HTML 문자열
    Returns:
        List[Dict]: 매장 정보 딕셔너리의 리스트
    """
    stores = []
    soup = BeautifulSoup(html, 'html.parser')
    
    # CSS 선택자로 table.tb_store > tbody > tr 선택
    rows = soup.select("article.altlist-body > ul > li")
    
    # find() 함수 이용 단일 태그로 찾기
    # table = soup.find("table", class_="tb_store")
    # tbody = table.find("tbody")
    # rows = tbody.find_all("tr")
   
    for tr in rows:
        tds = tr.find_all('div')
        if len(tds) < 6:
            continue
        
        store = {
            'region': tds[0].text.strip(),
            'name': tds[1].text.strip(),
            'status': tds[2].text.strip(),
            'address': tds[3].text.strip(),
            'service': ' '.join(img['alt'] for img in tds[4].find_all('img')),
            'phone': tds[5].text.strip()
        }
        stores.append(store)
    
    return stores

In [ ]:
# 테스트: 첫 페이지 파싱하기
stores = parse_store_info(html)
print(f"첫 페이지에서 찾은 매장 수: {len(stores)}")
print("\n첫 번째 매장 정보:")
pprint(stores[0])

## 3. 데이터 저장하기
- 수집한 데이터를 CSV와 JSON 형식으로 저장

In [ ]:
def save_to_files(stores: List[Dict], base_path: str = "./웹크롤링/커피매장"):
   """
   매장 정보를 CSV와 JSON 파일로 저장하는 함수
   """
   
   # 디렉토리 생성 (존재하지 않으면 생성)
   os.makedirs(base_path, exist_ok=True)
   
   # DataFrame 생성
   df = pd.DataFrame(stores)
   
   # CSV 파일로 저장
   file_name = "hollys_stores.csv"
   csv_file_path = os.path.join(base_path, file_name)
   df.to_csv(csv_file_path, encoding='utf-8', index=False)
   
   # JSON 파일로 저장 (DataFrame 활용)
   file_name = f"hollys_stores.json"
   json_file_path = os.path.join(base_path, file_name)
   df.to_json(json_file_path, orient='records', force_ascii=False, indent=4)
   
   print(f"CSV 파일 저장 완료: {csv_file_path}")
   print(f"JSON 파일 저장 완료: {json_file_path}")
   
   return df

In [ ]:
# 테스트: 첫 페이지 데이터 저장하기
df = save_to_files(stores)
display(df)

## 5. 전체 과정 실행하기

In [ ]:
stores = []

# 매장 정보 수집
print("할리스 매장 정보 수집 중...")
for page in range(1, 4):  # 1~3 페이지만 수집 (테스트용)
    html = get_hollys_store_info(page)
    if html:
        page_stores = parse_store_info(html)
        stores.extend(page_stores)
        print(f"페이지 {page}: {len(page_stores)}개의 매장 정보 수집 완료")

if not stores:
    print("매장 정보를 가져오는데 실패했습니다.")
    sys.exit(1)
    
print(f"\n총 {len(stores)}개의 매장 정보를 수집했습니다.")

# 데이터 저장
try:
    save_to_files(stores)
except Exception as e:
    print(f"데이터 저장 중 에러 발생: {e}")

In [166]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from datetime import datetime
from time import sleep

def get_article_list(page: int = 1, section_code: str = "S1N1") -> list:
    """
    AI타임즈 뉴스 목록 페이지에서 기사 정보 추출
    """
    base_url = "https://www.aitimes.com/news/articleList.html"
    params = {
        "page": str(page),
        "sc_section_code": section_code,  # AI 관련 섹션 코드
        "view_type": "sm"
    }
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                      'AppleWebKit/537.36 (KHTML, like Gecko) '
                      'Chrome/141.0.0.0 Safari/537.36'
    }

    response = requests.get(base_url, params=params, headers=headers)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "html.parser")
    articles = []

    # 기사 블록 선택
    for div in soup.select("div.altlist-webzine-content"):
        title_tag = div.select_one("h2.altlist-subject a")
        info_items = div.select("div.altlist-info-item")

        if not title_tag or len(info_items) < 2:
            continue

        title = title_tag.get_text(strip=True)
        url = title_tag["href"]
        reporter = info_items[0].get_text(strip=True)
        date = info_items[1].get_text(strip=True)

        articles.append({
            "title": title,
            "url": url,
            "reporter": reporter,
            "date": date
        })

    return articles


def get_article_content(url: str) -> str:
    """
    개별 기사 URL에서 본문 텍스트 추출
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                      'AppleWebKit/537.36 (KHTML, like Gecko) '
                      'Chrome/141.0.0.0 Safari/537.36'
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        content_div = soup.select_one("div.article-body")  # 본문 부분
        if not content_div:
            content_div = soup.select_one("div#article-view-content-div")

        content = content_div.get_text(separator="\n", strip=True) if content_div else ""
        return content
    except Exception as e:
        print(f"본문 수집 실패 ({url}): {e}")
        return ""


def crawl_ai_news(pages: int = 3):
    """
    여러 페이지에 걸쳐 AI 뉴스 기사 수집
    """
    all_articles = []

    for page in range(1, pages + 1):
        print(f"📄 {page}페이지 수집 중...")
        articles = get_article_list(page)

        for article in articles:
            article["content"] = get_article_content(article["url"])
            all_articles.append(article)
            sleep(1)  # 서버 과부하 방지

    return all_articles


def save_to_json(data: list, filename: str = "aitimes_ai_news.json"):
    """
    결과 데이터를 JSON 파일로 저장
    """
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"✅ {filename} 저장 완료 ({len(data)}개 기사)")


def summarize_articles(data: list):
    """
    날짜별 기사 수 등 간단한 통계 출력
    """
    df = pd.DataFrame(data)
    print("\n📊 날짜별 기사 수:")
    print(df["date"].value_counts())

    print("\n📰 최신 기사:")
    print(df.head(3)[["title", "date", "reporter"]])


if __name__ == "__main__":
    data = crawl_ai_news(pages=2)  # 원하는 페이지 수 조정
    save_to_json(data)
    summarize_articles(data)


📄 1페이지 수집 중...
본문 수집 실패 (): Invalid URL '': No scheme supplied. Perhaps you meant https://?
📄 2페이지 수집 중...
본문 수집 실패 (): Invalid URL '': No scheme supplied. Perhaps you meant https://?
✅ aitimes_ai_news.json 저장 완료 (42개 기사)

📊 날짜별 기사 수:
date
               2
10-27 18:00    1
10-27 16:40    1
10-27 13:05    1
10-24 17:00    1
10-24 15:52    1
10-23 16:54    1
10-22 12:44    1
10-29 15:45    1
10-02 14:09    1
10-01 06:00    1
09-26 10:05    1
09-30 18:11    1
09-16 18:50    1
09-15 16:50    1
09-15 16:14    1
09-18 18:30    1
09-13 14:50    1
09-13 10:20    1
09-11 15:40    1
09-10 16:05    1
09-09 17:00    1
09-09 12:27    1
09-08 16:30    1
09-08 16:05    1
09-05 16:00    1
09-04 14:50    1
09-03 16:15    1
09-01 14:50    1
08-29 17:05    1
08-29 16:35    1
08-29 12:20    1
08-28 11:41    1
08-25 10:50    1
08-24 16:25    1
08-22 16:58    1
08-22 16:00    1
08-20 07:35    1
08-19 16:30    1
08-17 16:45    1
08-15 07:26    1
Name: count, dtype: int64

📰 최신 기사:
                           

In [ ]:
import requests, json
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from time import sleep
import pandas as pd

BASE_URL = "https://www.aitimes.com"

def get_article_list(page: int = 1, section_code: str = "S1N1") -> list:
    """
    AI타임즈 뉴스 목록 페이지에서 기사 정보 추출
    """
    base_url = "https://www.aitimes.com/news/articleList.html"
    params = {
        "page": str(page),
        "sc_section_code": section_code,
        "view_type": "sm"
    }
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                      'AppleWebKit/537.36 (KHTML, like Gecko) '
                      'Chrome/141.0.0.0 Safari/537.36'
    }

    response = requests.get(base_url, params=params, headers=headers)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "html.parser")
    articles = []

    for div in soup.select("div.altlist-webzine-content"):
        title_tag = div.select_one("h2.altlist-subject a")
        info_items = div.select("div.altlist-info-item")

        if not title_tag or len(info_items) < 2:
            continue

        title = title_tag.get_text(strip=True)
        relative_url = title_tag["href"]
        url = urljoin(BASE_URL, relative_url)   # ✅ 절대경로 변환
        reporter = info_items[0].get_text(strip=True)
        date = info_items[1].get_text(strip=True)

        articles.append({
            "title": title,
            "url": url,
            "reporter": reporter,
            "date": date
        })

    return articles


def get_article_content(url: str) -> str:
    """
    개별 기사 URL에서 본문 텍스트 추출
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                      'AppleWebKit/537.36 (KHTML, like Gecko) '
                      'Chrome/141.0.0.0 Safari/537.36'
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        # ✅ 본문 선택자 수정
        paragraphs = soup.select("article.article-veiw-body.view-page.font-size18 p")
        content = "\n".join(p.get_text(strip=True) for p in paragraphs)

        return content
    except Exception as e:
        print(f"본문 수집 실패 ({url}): {e}")
        return ""


def crawl_ai_news(pages: int = 3):
    all_articles = []

    for page in range(1, pages + 1):
        print(f"📄 {page}페이지 수집 중...")
        articles = get_article_list(page)

        for article in articles:
            article["content"] = get_article_content(article["url"])
            all_articles.append(article)
        sleep(0.
              1)

    return all_articles


def save_to_json(data: list, filename: str = "aitimes_ai_news.json"):
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"✅ {filename} 저장 완료 ({len(data)}개 기사)")


def summarize_articles(data: list):
    df = pd.DataFrame(data)
    print("\n📊 날짜별 기사 수:")
    print(df["date"].value_counts())

    print("\n📰 최신 기사:")
    print(df.head(3)[["title", "date", "reporter"]])


if __name__ == "__main__":
    data = crawl_ai_news(pages=2)
    save_to_json(data)
    summarize_articles(data)


📄 1페이지 수집 중...
📄 2페이지 수집 중...
✅ aitimes_ai_news.json 저장 완료 (42개 기사)

📊 날짜별 기사 수:
date
               2
10-27 18:00    1
10-27 16:40    1
10-27 13:05    1
10-24 17:00    1
10-24 15:52    1
10-23 16:54    1
10-22 12:44    1
10-29 15:45    1
10-02 14:09    1
10-01 06:00    1
09-26 10:05    1
09-30 18:11    1
09-16 18:50    1
09-15 16:50    1
09-15 16:14    1
09-18 18:30    1
09-13 14:50    1
09-13 10:20    1
09-11 15:40    1
09-10 16:05    1
09-09 17:00    1
09-09 12:27    1
09-08 16:30    1
09-08 16:05    1
09-05 16:00    1
09-04 14:50    1
09-03 16:15    1
09-01 14:50    1
08-29 17:05    1
08-29 16:35    1
08-29 12:20    1
08-28 11:41    1
08-25 10:50    1
08-24 16:25    1
08-22 16:58    1
08-22 16:00    1
08-20 07:35    1
08-19 16:30    1
08-17 16:45    1
08-15 07:26    1
Name: count, dtype: int64

📰 최신 기사:
                                              title         date reporter
0                과기부, APEC 한-미 정상회담서 AI기술 협력 MOU 체결  10-29 15:45   장세민 기자
1  APEC CEO 서밋에 글로벌 기업인 1700명 참여.